# Predictive Modelling Task - 2

In this workbook, I will process all stocks provided in the dataset using the process-pipeline I developed in task-1.

First, I will wrap the forecasting routine into a function:

In [2]:
import pandas as pd
import numpy as np
import warnings
# Dickey-Fuller Test
from statsmodels.tsa.stattools import adfuller
# auto corelation
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
# seasonal decompose
from statsmodels.tsa.seasonal import seasonal_decompose
# ARIMA
import statsmodels.tsa as tsa
# display
from IPython.display import display

warnings.filterwarnings('ignore')

# plot utilities
%matplotlib inline
import matplotlib.pyplot as plt
import mpld3
mpld3.enable_notebook()

def testStationarity(ts=None, displaySummary=True, displayPlot=True):
    ''' Test whether the input series is stationary or not
    '''
    # remove NAN's
    ts = ts.dropna()
    
    # create a data frame for X1
    dframe = pd.DataFrame(ts.index)
    dframe['P(t)'] = ts.values
    d = dframe['P(t)'][~dframe['P(t)'].isnull()]

    # dickyey-fuller test
    df = adfuller(ts)
    if df[0] < df[4]['1%']:
        confi = 0.99
        isStationary = True;
        strStationary = ' DFTest: Stationary'+ ' (confidence= %.2f)' % confi
        
    elif df[0] < df[4]['5%']:
        confi = 0.95
        isStationary = True;
        strStationary = ' DFTest: Stationary'+ ' (confidence= %.2f)' % confi
        
    elif df[0] < df[4]['10%']:
        confi = 0.90
        isStationary = True;
        strStationary = ' DFTest: Stationary' + ' (confidence= %.2f)' % confi
        
    else:
        confi = 0
        isStationary = False;
        strStationary = ' DFTest: Non Stationary'
        
    # moving averages
    dframe['MA_30'] = pd.Series.rolling(d, center=False, window=30).mean() # 30 -day MA
    dframe['MA_60'] = pd.Series.rolling(d, center=False, window=60).mean() # 60 -day MA
    dframe['MA_120'] = pd.Series.rolling(d, center=False, window=120).mean() # 120 -day MA

    # standard devs
    dframe['SD_30'] = pd.Series.rolling(d, center=False, window=30).std() # 30 -day SD
    dframe['SD_60'] = pd.Series.rolling(d, center=False, window=60).std() # 60 -day SD
    dframe['SD_120'] = pd.Series.rolling(d, center=False, window=120).std() # 120 -day SD
    
    if displayPlot:
        # plot
        dframe.loc[:, ['MA_30', 'MA_60', 'MA_120']].plot( title='Moving Averages - ' + ts.name + strStationary)
        dframe.loc[:, ['SD_30', 'SD_60', 'SD_120']].plot( title='Variance - ' + ts.name + strStationary)
        
    if displaySummary:
        print(df)
        print('Time series - '+ts.name + ',' + strStationary)
        
    return isStationary


def convert2Stationary(ts=None, disp=True):
    ''' Convert a non-stationary time-series to stationary using simple differencing of
        log-transformed timeseries
        The input time-series is first checked for stationarity then the conversion is done
        
        input: ts -> time-series in normal price
        output: stationary timeseries
    '''
    # log
    ts_log = np.log(ts.dropna())
    out = ts_log
    if not testStationarity(ts, displayPlot=False, displaySummary=False):
        # differencing
        out = ts_log - ts_log.shift(1)
    
    # perform test
    if disp:
        print("=== after conversion test for Stationarity===")
        testStationarity(out, displayPlot=disp, displaySummary=disp)
        
    return out



def forecast_3day_returns(ts=None, ARIMAorder=None):
    ''' Forecast 3-day returns using ARIMA modelling
    '''
    # original timeseries
    orig_tseries = ts
    # create a new time series by pushing it 3-days ahead and fill it with 0
    pushed_tseries = pd.Series(0, orig_tseries.index + pd.Timedelta('3 day'))
    # extract the last 3-days from pushed_tseries and append it to original timeseries
    tseries = orig_tseries.append(pushed_tseries[-3 : pushed_tseries.shape[0]], verify_integrity=True)

    # convert original timeseries X3 to stationary
    day_returns = convert2Stationary(ts, False)
    # index-0 of day_returns will be NAN due to differencing.
    # usually day_returns[original_length+1] will have -INF due to 0 insertion.

    if ARIMAorder is None:
        ARIMAorder = (1,0,1)
        
    # modelling (use all data starting from index-1 day of day_returns to the lastday of the orignal timeseries)
    model = tsa.arima_model.ARIMA(day_returns.ix[1:], ARIMAorder).fit(disp=-1)
    print(model.summary())

    # index of day_returns runs from 0:day_returns.shape[0]-1, there are total of day_returns.shape[0] elements
    # we are ignoring inedx-0 while training (it might contain NAN due to differencing), hence we are only training
    # with day_returns.shape[0]-1 elements (with indexing running from 1:day_returns.shape[0]-1). 
    # Therefore the the trained-model will have a length of ((day_returns.shape[0]-1)-1)
    # For prediction, the starting index should atleast be the last fittedvalue.

    # predict
    preds = model.predict(model.fittedvalues.shape[0]-1, model.fittedvalues.shape[0]+2 )

    ### calculate error metrics and accuracy of fittedvalues
    # Original-returns: calculate rolling_sum of log-returns over a window of 4 days, including current day
    orig_returns = day_returns[1:]
    rollsum = pd.Series.rolling(orig_returns, center=False, window=4).sum()
    orig_3day_returns =  orig_returns - rollsum.shift(-3)
    # fitted-returns: 
    rollsum = pd.Series.rolling(model.fittedvalues, center=False, window=4).sum()
    fitted_3day_returns =  model.fittedvalues - rollsum.shift(-3)
    # display dataframe

    statsFrame = pd.DataFrame(orig_returns.index)
    statsFrame['Orig Log Returns'] = orig_returns.values
    statsFrame['Pred Log Returns'] = model.fittedvalues.values
    statsFrame['AbsError'] = np.abs(orig_returns.values - model.fittedvalues.values)
    statsFrame['AbsPercentError(%)'] = np.abs((orig_returns.values - model.fittedvalues.values)/(orig_returns.values+0.00000001)) * 100 
    statsFrame['DirectionAccuracy'] = ~((orig_returns.values > 0) != (model.fittedvalues.values > 0))
    statsFrame['Orig 3-day Log returns'] = orig_3day_returns.values
    statsFrame['Pred 3-day Log returns'] = fitted_3day_returns.values
    statsFrame['3-day AbsError'] = np.abs(orig_3day_returns.values - fitted_3day_returns.values)
    statsFrame['3-day AbsPercentError(%)'] = np.abs((orig_3day_returns.values - fitted_3day_returns.values)/(orig_3day_returns.values+0.00000001)) * 100 
    statsFrame['3-day DirectionAccuracy'] = ~((orig_3day_returns.values > 0) != (fitted_3day_returns.values > 0))
    # calculate


    # create a fitted_day_returns series using index from new timeseries (created used pushed_timeseries) and fittedvalues
    fitted_day_returns = pd.Series(model.fittedvalues, index=tseries.index)
    # copy the forecast values to correct dates
    fitted_day_returns[-4 : fitted_day_returns.shape[0]] = preds
    forecast_day_returns = fitted_day_returns[-4 : fitted_day_returns.shape[0]]

    # Predicted-returns: 
    rollsum = pd.Series.rolling(forecast_day_returns, center=False, window=4).sum()
    forecast_3day_returns =  forecast_day_returns - rollsum.shift(-3)

    # Display
    print("=====  3-day Returns forecast for closing price of last-day in timeseries ======")
    print("           Note: The first row in the table corresponds to the last day \n           closing price of the stock")
    print("   Stock: "+ ts.name)
    futureFrame = pd.DataFrame(forecast_day_returns.index)
    futureFrame['Forecast Log Returns'] = forecast_day_returns.values
    futureFrame['Forecast 3-day Log Returns'] = forecast_3day_returns.values
    futureFrame['Long term MAE (log returns)'] = np.mean(statsFrame['AbsError'])
    futureFrame['Long term MAPE (log returns) %'] = np.mean(statsFrame['AbsPercentError(%)']/100)*100
    futureFrame['Long term DirectionAccuracy (log returns) %'] = (sum(statsFrame['DirectionAccuracy'])/statsFrame.shape[0]) * 100
    futureFrame['Long term MAE (3-day log returns)'] = np.mean(statsFrame['3-day AbsError'])
    futureFrame['Long term MAPE (3-day log returns) %'] = np.mean(statsFrame['3-day AbsPercentError(%)']/100) * 100 
    futureFrame['Long term DirectionAccuracy (3-day log returns) %'] = (sum(statsFrame['3-day DirectionAccuracy'])/statsFrame.shape[0]) * 100
    display(futureFrame)
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
    
    
    return futureFrame



# test- forecast
# convert data to time-series data while reading from csv itself
dailyPrices = pd.read_csv('dailyPrices_AtClose.csv', parse_dates='t()', index_col='t()')
#dailyPrices.index # check if the index is datetime
# re-index and set frequency='D'
dates = pd.date_range(dailyPrices.index[0], dailyPrices.index[-1], freq='D')
dailyPrices = dailyPrices.reindex(dates)
#dailyPrices.index

forecast_3day_returns(dailyPrices.X3)
    


                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 1)   Log Likelihood                6734.171
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13460.342
Time:                        19:58:13   BIC                         -13436.639
Sample:                    01-04-2005   HQIC                        -13451.781
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0003      0.000      0.772      0.440        -0.000     0.001
ar.L1.X3       0.5033      0.112      4.487      0.000         0.283     0.723
ma.L1.X3      -0.5830      0.105     -5.553      0.0

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,-0.000133,-0.001179,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
1,2016-01-01,0.000486,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
2,2016-01-02,0.000374,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
3,2016-01-03,0.000318,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,-0.000133,-0.001179,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
1,2016-01-01,0.000486,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
2,2016-01-02,0.000374,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
3,2016-01-03,0.000318,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948


### Process All Stocks

I will now loop through all stocks and present the results.

In [3]:
# process
for stock in dailyPrices:
    print('*******************************    Stock :  ' + dailyPrices[stock].name + '   ****************************')
    forecast_3day_returns(dailyPrices[stock], ARIMAorder=(1,0,0))
    


*******************************    Stock :  X1   ****************************
                              ARMA Model Results                              
Dep. Variable:                     X1   No. Observations:                  977
Model:                     ARMA(1, 0)   Log Likelihood                2927.165
Method:                       css-mle   S.D. of innovations              0.012
Date:                Wed, 08 Jun 2016   AIC                          -5848.330
Time:                        19:58:18   BIC                          -5833.676
Sample:                    01-04-2005   HQIC                         -5842.754
                         - 11-18-2008                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0004      0.000      1.201      0.230        -0.000     0.001
ar.L1.X1      -0.1273      0.032     -4.013      0.00

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000316,-0.001284,0.008147,90475.878952,53.224156,0.014308,33165.12993,36.745138
1,2016-01-01,0.000465,NaN,0.008147,90475.878952,53.224156,0.014308,33165.12993,36.745138
2,2016-01-02,0.000406,NaN,0.008147,90475.878952,53.224156,0.014308,33165.12993,36.745138
3,2016-01-03,0.000413,NaN,0.008147,90475.878952,53.224156,0.014308,33165.12993,36.745138


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
*******************************    Stock :  X2   ****************************
                              ARMA Model Results                              
Dep. Variable:                     X2   No. Observations:                 1008
Model:                     ARMA(1, 0)   Log Likelihood                2749.797
Method:                       css-mle   S.D. of innovations              0.016
Date:                Wed, 08 Jun 2016   AIC                          -5493.594
Time:                        19:58:18   BIC                          -5478.847
Sample:                    01-04-2005   HQIC                         -5487.992
                         - 01-05-2009                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0006      0.000      1.150      0.

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000525,-0.001672,0.009607,72649.480731,50.892857,0.017149,172515.825677,51.190476
1,2016-01-01,0.000573,NaN,0.009607,72649.480731,50.892857,0.017149,172515.825677,51.190476
2,2016-01-02,0.000549,NaN,0.009607,72649.480731,50.892857,0.017149,172515.825677,51.190476
3,2016-01-03,0.000550,NaN,0.009607,72649.480731,50.892857,0.017149,172515.825677,51.190476


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
*******************************    Stock :  X3   ****************************
                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 0)   Log Likelihood                6727.408
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13448.817
Time:                        19:58:18   BIC                         -13431.039
Sample:                    01-04-2005   HQIC                        -13442.396
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0003      0.000      0.684      0.

...

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
*******************************    Stock :  X55   ****************************
                              ARMA Model Results                              
Dep. Variable:                    X55   No. Observations:                  798
Model:                     ARMA(1, 0)   Log Likelihood                2059.383
Method:                       css-mle   S.D. of innovations              0.018
Date:                Wed, 08 Jun 2016   AIC                          -4112.767
Time:                        19:58:31   BIC                          -4098.720
Sample:                    10-31-2012   HQIC                         -4107.370
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0011      0.001      1.560      0

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000304,-0.003287,0.013369,115165.106711,51.378446,0.023429,85994.674145,56.641604
1,2016-01-01,0.001171,NaN,0.013369,115165.106711,51.378446,0.023429,85994.674145,56.641604
2,2016-01-02,0.001060,NaN,0.013369,115165.106711,51.378446,0.023429,85994.674145,56.641604
3,2016-01-03,0.001056,NaN,0.013369,115165.106711,51.378446,0.023429,85994.674145,56.641604


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
*******************************    Stock :  X56   ****************************
                              ARMA Model Results                              
Dep. Variable:                    X56   No. Observations:                  946
Model:                     ARMA(1, 0)   Log Likelihood                2737.477
Method:                       css-mle   S.D. of innovations              0.013
Date:                Wed, 08 Jun 2016   AIC                          -5468.955
Time:                        19:58:31   BIC                          -5454.398
Sample:                    01-04-2005   HQIC                         -5463.408
                         - 10-06-2008                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0005      0.000      1.117      0

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000471,-0.001417,0.007754,61426.969564,50.634249,0.013794,174214.406922,52.536998
1,2016-01-01,0.000455,NaN,0.007754,61426.969564,50.634249,0.013794,174214.406922,52.536998
2,2016-01-02,0.000481,NaN,0.007754,61426.969564,50.634249,0.013794,174214.406922,52.536998
3,2016-01-03,0.000481,NaN,0.007754,61426.969564,50.634249,0.013794,174214.406922,52.536998


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
